Antes de iniciar importação de planilhas, instalar todas as dependencias que constam no arquivo 'requirements.txt', onde:<br>
pandas - biblioteca que realiza leitura e modela as planilhas<br>
sqlalchemy - biblioteca de banco de dados<br>
psycopg2 - biblioteca complementar de banco de dados postgresql<br>
sshtunnel - estabelece tunel de conexão com servidor via vpn<br>
obs: necessário estar conectado via vpn com banco de dados

In [ ]:
#importação de planilhas
import psycopg2
import pandas as pd
from sshtunnel import SSHTunnelForwarder
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine

In [ ]:
#cria o tunel de conexao ssh com o servidor
tunnel = SSHTunnelForwarder(
    ('143.54.25.131', 22),
    ssh_username="atlas-oportunidades-srv01",
    ssh_password = "Mt0c5P!lzT",
    remote_bind_address=('localhost', 5432),
    local_bind_address=('localhost',8000), # could be any available port
)
# inicializacao do tunel
tunnel.start()

#declaracao de variavel de conexao com banco de dados
conn = create_engine(f'postgresql://atlas_adm:123456@{tunnel.local_bind_host}:{tunnel.local_bind_port}/atlasdb')

In [ ]:
#realiza a leitura da planilha de dados e salva em um dataframe
df = pd.read_excel("./Data/_State/Data/Dados - N3.xlsx")
#porem mantem apenas essas duas colunas
colunas_manter = ['code', 'name']
df = df.loc[:, colunas_manter]
#renomeia as colunas para as colunas de acordo com tabela tb_estados
df = df.rename(columns={'code': 'cd_estado', 'name': 'nm_estado'})

In [ ]:
#inicia conexao com banco de dados e envia os dados para o servidor
dbConnection = conn.connect()
df.to_sql(    
    "tb_estados",
    dbConnection,
    schema="atlas_schema",
    if_exists='replace',
    index = False
)
dbConnection.close()

In [ ]:
# realiza a leitura das tabelas agencia, unidades, formato e classificacao
# salva em dataframes para comparações 
dbConnection = conn.connect()
df_agencia = pd.read_sql("select * from atlas_schema.tb_agencias", dbConnection)
df_unidade = pd.read_sql("select * from atlas_schema.tb_unidades", dbConnection)
df_formato = pd.read_sql("select * from atlas_schema.tb_formato", dbConnection)
df_classificacao = pd.read_sql("select * from atlas_schema.tb_classificacao", dbConnection)
dbConnection.close()

In [ ]:
# realiza a leitura da planilha de dicionario de estados
# mantem só algumas colunas
df = pd.read_excel("./Data/_State/dictionary_state.xlsx")
colunas_manter = ['Agency', 'Name', 'Description', 'Descrição', 'Label', 'Rótulo', 'Unit of analysis', 'Format', 'Classificação']
df = df.loc[:, colunas_manter]
df = df.rename(
    columns={
        'Name': 'cd_nm_coluna', 
        'Agency': 'nm_agencia',
        'Description': 'nm_descricao_en', 
        'Descrição': 'nm_descricao_pt', 
        'Label': 'nm_label_en', 
        'Rótulo': 'nm_label_pt', 
        'Unit of analysis': 'nm_unidade', 
        'Format': 'nm_formato', 
        'Classificação': 'nm_classificacao_pt'
    }
)

In [ ]:
# a partir da leitura das tabelas de agencia, unidade, formato e classificacao em um dos blocos anteriores
# é feito o merge no dataframe atual para assim obter a chave id das mesmas na tabela de dicionario
df_final = pd.merge(df, df_agencia, on='nm_agencia', how='left')
df_final = pd.merge(df_final, df_unidade, on='nm_unidade', how='left')
df_final = pd.merge(df_final, df_formato, on='nm_formato', how='left')
df_final = pd.merge(df_final, df_classificacao, on='nm_classificacao_pt', how='left')

In [ ]:
# uma vez que o merhe foi feito não é necessário algumas colunas pois já existem nas tabelas agencia, unidade, formato e classificacao
# então é feito a exclusão
colunas_manter = ['cd_nm_coluna', 'nm_descricao_en', 'nm_descricao_pt', 'nm_label_en', 'nm_label_pt', 'cd_agencia', 'cd_unidade', 'cd_formato', 'cd_classificacao']
df_final = df_final.loc[:, colunas_manter]

In [ ]:
# após realizar todos os ajustes, os dados são salvos no banco de dados
dbConnection = conn.connect()
df_final.to_sql(    
    "tb_dicionario",
    dbConnection,
    schema="atlas_schema",
    if_exists='append',
    index = False
)
dbConnection.close()

In [ ]:
# iniciando leitura da planilha de dados N3
df = pd.read_excel("./Data/_State/Data/Dados - N3.xlsx")
# excluindo coluna de nome 'name'
df = df.drop('name', axis=1)

In [ ]:
# as colunas são transformadas em dados
# como um pivot table
df1 = df.melt(
    id_vars=['code'],
    var_name='cd_nm_coluna',
    value_name='vl_por_cd'
).sort_values(by='code')

# coluna renomeada
df1 = df1.rename(columns={'code':'cd_estado'})

In [ ]:
# apos modelagem dos dados é feito o envio para banco de dados
dbConnection = conn.connect()
df1.to_sql(    
    "tb_cod_valor_estado",
    dbConnection,
    schema="atlas_schema",
    if_exists='append',
    index = False
)
dbConnection.close()

In [ ]:
# iniciando leitura da planilha de dados IPEA
df = pd.read_excel("./Data/_State/Data/IPEA - Dados Estaduais.xlsx")

# excluindo coluna de nome 'name'
df = df.drop('name', axis=1)

In [ ]:
# as colunas são transformadas em dados
# como um pivot table
df1 = df.melt(
    id_vars=['code'],
    var_name='cd_nm_coluna',
    value_name='vl_por_cd'
).sort_values(by='code')

# coluna renomeada
df1 = df1.rename(columns={'code':'cd_estado'})

In [ ]:
# apos modelagem dos dados é feito o envio para banco de dados
dbConnection = conn.connect()
df1.to_sql(    
    "tb_cod_valor_estado",
    dbConnection,
    schema="atlas_schema",
    if_exists='append',
    index = False
)
dbConnection.close()